In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read malware data
malware_data = pd.read_csv("CSDMC_API_Train.csv")


In [2]:
# TODO: Calculate number of malware traces
n_traces = len(malware_data)

# TODO: Calculate number of features
n_features = len(malware_data.columns) - 1 


print malware_data.columns
target = malware_data['Type']

# Print the results
print "Total number of traces: {}".format(n_traces)
print "Total number of types: {}".format(len(target))


Index([u'Type', u' APICalls'], dtype='object')
Total number of traces: 388
Total number of types: 388


In [3]:
Apicalls = malware_data[' APICalls']
print Apicalls.head()
# Print the results
print "Total number of APICalls list: {}".format(len(Apicalls))

0    LoadLibraryW HeapAlloc HeapAlloc HeapFree Heap...
1    RegOpenKeyExW LoadLibraryA GetProcAddress GetP...
2    HeapAlloc HeapFree HeapAlloc HeapAlloc HeapFre...
3    HeapAlloc HeapFree HeapAlloc HeapAlloc HeapFre...
4    HeapAlloc HeapFree HeapAlloc HeapAlloc HeapFre...
Name:  APICalls, dtype: object
Total number of APICalls list: 388


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(Apicalls)
X_train_counts.shape
#count_vect.vocabulary_.get(u'heapalloc')

(388, 313)

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape
print X_train_tfidf[6]

  (0, 173)	0.00543723504589
  (0, 128)	0.331095095537
  (0, 131)	0.399416305727
  (0, 199)	0.0198599681146
  (0, 96)	0.0767630770601
  (0, 58)	0.00875912951155
  (0, 201)	0.00794398724586
  (0, 237)	0.00635196166572
  (0, 174)	0.00180275511264
  (0, 108)	0.00456717955445
  (0, 75)	0.0270597553152
  (0, 63)	0.00435021593264
  (0, 44)	0.0042762422896
  (0, 50)	0.00437569819515
  (0, 289)	0.00950612831956
  (0, 236)	0.0173782290518
  (0, 242)	0.0326822663997
  (0, 216)	0.0243156226668
  (0, 175)	0.00180747100736
  (0, 243)	0.00667078494466
  (0, 56)	0.00370117487383
  (0, 286)	0.00743742616175
  (0, 287)	0.00437569819515
  (0, 85)	0.00363761573359
  (0, 14)	0.0255141916654
  :	:
  (0, 120)	0.00812395456202
  (0, 72)	0.0111476942224
  (0, 277)	0.00308739332903
  (0, 271)	0.00278015125777
  (0, 129)	0.00453829946357
  (0, 21)	0.00802378968983
  (0, 261)	0.0216796540102
  (0, 257)	0.0138446053401
  (0, 69)	0.0095727753509
  (0, 1)	0.0470478052843
  (0, 2)	0.0451644095434
  (0, 0)	0.006262291

In [45]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),])

In [46]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Apicalls, target, test_size=.2, random_state =1)

In [47]:
text_clf = text_clf.fit(X_train, y_train)

In [48]:
predicted = text_clf.predict(X_test)

In [49]:
np.mean(predicted == y_test)

0.84615384615384615

In [42]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5, random_state=42)),])

In [43]:
text_clf = text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.89743589743589747

In [44]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted,target_names=['Benign','Malware']))

             precision    recall  f1-score   support

     Benign       0.83      0.42      0.56        12
    Malware       0.90      0.98      0.94        66

avg / total       0.89      0.90      0.88        78

